# Coursera Capstone


This repository is mainly used to implement the coursera capstone project.

The Capstone project will help us implement the knowledge obtained from the previous courses.

**Importing Necessary Libraries**

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# import the library we use to open URLs
import urllib.request

# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

**Downloading Data and processing Data**

In [6]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page)
table = soup.findAll("table", {"class": "wikitable sortable"})

In [7]:
A=[]
B=[]
C=[]

for row in table[0].find_all('tr'):
    cells=row.find_all('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

**Removing \n from the data**

In [8]:
A1 =[]
B1=[]
C1=[]

for a in A:
    A1.append(a.split('\n')[0])
for b in B:
    B1.append(b.split('\n')[0])
for c in C:
    C1.append(c.split('\n')[0])

A = A1
B = B1
C = C1

**Creating the DataFrame

In [9]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods['PostalCode'] = A
neighborhoods['Borough'] = B
neighborhoods['Neighborhood'] = C
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Removing Borough with Not assigned as value**

In [10]:
neighborhoods = neighborhoods[neighborhoods['Borough'] != 'Not assigned']
neighborhoods = neighborhoods.reset_index(drop=True)
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**Checking for any neighbourhood with not assigned as value**

In [11]:
neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


**Shape of the dataframe**

In [12]:
neighborhoods.shape

(103, 3)

**Reading CSV File With Coordinates**

In [1]:
df = data = pd.read_csv("Geospatial_Coordinates.csv")
df.head()

NameError: name 'pd' is not defined

**Appeding Both the Dataframes into a Single DataFrame**

In [ ]:
Latitude = []
Longitude = []
for pc in neighborhoods['PostalCode']:
    dfRow = df[df['Postal Code']==pc]
    Latitude.append(dfRow['Latitude'].values[0])
    Longitude.append(dfRow['Longitude'].values[0])

In [ ]:
neighborhoods = neighborhoods.assign(Latitude = Latitude)

In [ ]:
neighborhoods= neighborhoods.assign(Longitude = Longitude)

In [ ]:
neighborhoods.shape

In [ ]:
neighborhoods